In [65]:
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import DataLoader,TensorDataset
import torch.optim as optim
import numpy as np
import torch.nn as nn

In [66]:
data=datasets.load_breast_cancer()

In [67]:
df=pd.DataFrame(data=data.data,columns=data.feature_names)
df['target']=data.target

In [68]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [69]:
df['target'].value_counts()

target
1    357
0    212
Name: count, dtype: int64

In [70]:
df['target'].unique()

array([0, 1])

#0==>BENIGN

#1==>MALIGNANT

In [71]:
df.isnull().sum()

mean radius                0
mean texture               0
mean perimeter             0
mean area                  0
mean smoothness            0
mean compactness           0
mean concavity             0
mean concave points        0
mean symmetry              0
mean fractal dimension     0
radius error               0
texture error              0
perimeter error            0
area error                 0
smoothness error           0
compactness error          0
concavity error            0
concave points error       0
symmetry error             0
fractal dimension error    0
worst radius               0
worst texture              0
worst perimeter            0
worst area                 0
worst smoothness           0
worst compactness          0
worst concavity            0
worst concave points       0
worst symmetry             0
worst fractal dimension    0
target                     0
dtype: int64

In [72]:
df.shape

(569, 31)

In [73]:
x=df.iloc[:,0:30]

In [74]:
x.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [75]:
y=df['target']

In [76]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

In [77]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=11,stratify=y)

In [78]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((455, 30), (114, 30), (455,), (114,))

In [79]:
ss=StandardScaler()
x_train=ss.fit_transform(x_train)
x_test=ss.transform(x_test)

In [80]:
#convert tensor into vector
x_train_tensor=torch.tensor(x_train,dtype=torch.float32)
x_test_tensor=torch.tensor(x_test,dtype=torch.float32)
y_train_tensor=torch.tensor(y_train.values,dtype=torch.float32)
y_test_tensor=torch.tensor(y_test.values,dtype=torch.float32)

In [81]:
type(x_train_tensor)

torch.Tensor

In [82]:
class Binary(nn.Module):
    def __init__(self,new_features):
        super().__init__()
        self.linear1=nn.Linear(new_features,60)
        self.relu1=nn.ReLU()
        self.linear2=nn.Linear(60,90)
        self.relu2=nn.ReLU()
        self.linear3=nn.Linear(90,60)
        self.relu3=nn.ReLU()
        self.linear4=nn.Linear(60,30)
        self.relu4=nn.ReLU()
        self.linear5=nn.Linear(30,15)
        self.relu5=nn.ReLU()
        self.linear6=nn.Linear(15,1)
        self.sigmoid=nn.Sigmoid()

    def forward(self,data):
        x=self.linear1(data)
        x=self.relu1(x)
        x=self.linear2(x)
        x=self.relu2(x)
        x=self.linear3(x)
        x=self.relu3(x)
        x=self.linear4(x)
        x=self.relu4(x)
        x=self.linear5(x)
        x=self.relu5(x)
        x=self.linear6(x)
        x=self.sigmoid(x)
        return x

In [83]:
x_train_tensor.shape[1]

30

In [84]:
model=Binary(x_train_tensor.shape[1])

In [118]:
y_test_tensor.size(0)

114

In [85]:
x_train_dataset=TensorDataset(x_train_tensor,y_train_tensor)
x_test_dataset=TensorDataset(x_test_tensor,y_test_tensor)

In [86]:
train_loader=DataLoader(x_train_dataset,batch_size=30,shuffle=True)
test_loader=DataLoader(x_test_dataset,batch_size=30,shuffle=False)

In [87]:
learning_rate=0.01

In [88]:
criterion=nn.BCELoss()

In [89]:
optimizer=optim.Adam(model.parameters(),learning_rate)

In [90]:
def neuralnetwork(model,train_loader,criterion,optimizer,epochs=20):
    for epoch in range(epochs):
        model.train()#there are two modes we are putting our model to training mode 
        running_loss=0
        for input,label in train_loader:
            #forward pass
            prediction=model(input)
            label=label.unsqueeze(1)

            #loss calculation
            loss=criterion(prediction,label)

            optimizer.zero_grad()

            #backward pass and update weight
            loss.backward()
            optimizer.step()

            running_loss +=loss.item()
        print(f"epochs are {epoch+1}/{epochs} and the loss are {running_loss/len(train_loader)}")


In [91]:
neuralnetwork(model,train_loader,criterion,optimizer,30)

epochs are 1/30 and the loss are 0.3602696666494012
epochs are 2/30 and the loss are 0.33298213228044915
epochs are 3/30 and the loss are 0.049622672458291106
epochs are 4/30 and the loss are 0.037418117935885675
epochs are 5/30 and the loss are 0.03358641352679115
epochs are 6/30 and the loss are 0.024396123524638824
epochs are 7/30 and the loss are 0.018182668999770613
epochs are 8/30 and the loss are 0.011273986202121833
epochs are 9/30 and the loss are 0.02433805946202483
epochs are 10/30 and the loss are 0.012719063374447614
epochs are 11/30 and the loss are 0.010414587167247191
epochs are 12/30 and the loss are 0.005979904067118014
epochs are 13/30 and the loss are 0.004914777890377309
epochs are 14/30 and the loss are 0.001393912654975793
epochs are 15/30 and the loss are 0.000518298436164244
epochs are 16/30 and the loss are 0.00024304895448579167
epochs are 17/30 and the loss are 0.00012659928428948586
epochs are 18/30 and the loss are 9.012724189805255e-05
epochs are 19/30 an

In [122]:
op=model(x_test_tensor[0])

In [126]:
result=(op>0.5).float()
print(result)

tensor([0.])


In [135]:
(op==y_test_tensor[0]).sum().item()

0

In [119]:
def evaluation(model,test_loader):
    model.eval()
    correct=0
    total=0
    with torch.no_grad():
        for input,label in test_loader:
            label=label.unsqueeze(1)
            output=model(input)
            prediction=(output>0.5).float()
            total+=label.size(0)
            correct +=(prediction==label).sum().item()
        print("correct is ",correct)
        print("total is",total)
        accuracy=correct/total
        print(f"total accuracy accuracy : {accuracy*100}")
            

In [136]:
110/114

0.9649122807017544

In [120]:
evaluation(model,test_loader)

correct is  110
total is 114
total accuracy accuracy : 96.49122807017544
